In [43]:
import requests
import pandas as pd
import json
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from tqdm import tqdm_notebook as tqdm
import pickle

In [38]:
today = datetime(2019,12,31)
date_list = [(today - timedelta(days = x)).strftime("%Y%m%d") for x in range(365)]

In [40]:
origo_dict = {}
for date in tqdm(date_list):
    response = requests.get(f'https://www.origo.hu/hir-archivum/2019/{date}.html')
    soup = BeautifulSoup(response.text, 'html.parser')
    articles = [article.get("href") for article in soup.select(".archive-cikk a")]
    for article in articles:
        response = requests.get(article)
        soup = BeautifulSoup(response.text, 'html.parser')
        try:
            sources = [source.text.split("Forrás: ")[1] for source in soup.select(".source")]
        except:
            sources = []
        try:
            address_top = soup.find('div', class_='address top')
            author = address_top.find("span", {"class":"article-author"}).text
            date = address_top.find("span", {"id":"article-date"}).text
        except:
            author = None
            date = None

        paragraphs = soup.select("#article-text p")
        related_urls = []
        for paragraph in paragraphs:
            if "MTI" in paragraph.text or "mti" in paragraph.text or "Mti" in paragraph.text:
                related_urls.append("MTI")
            if paragraph.find_all("a") is not None:
                for link in paragraph.find_all("a"):
                    if "http" in link.get('href'):
                        related_urls.append(link.get('href'))
                    else:
                        related_urls.append("https:" + link.get('href'))
        origo_dict[article] = {
            "related_links" : related_urls,
            "sources" : sources,
            "author" : author,
            "publish_date" : date
            #"last_modification_date" : dik[0]["_mod_ts"]
        }

In [41]:
len(origo_dict)

53370

In [44]:
with open('origo.p', 'wb') as f:
    pickle.dump(origo_dict, f)

In [49]:
with open('origo.p', 'rb') as handle:
    b = pickle.load(handle)

In [51]:
b

{'https://www.origo.hu/auto/20191228-roncsderbi-egy-birsag-miatt.html': {'related_links': [],
  'sources': [],
  'author': 'Origo',
  'publish_date': '2019.12.31. 08:42'},
 'https://www.origo.hu/filmklub/20191230-a-magyar-film-2019ben.html': {'related_links': ['https://www.origo.hu/filmklub/20190830-meghalt-toth-janos-operator-a-nemzet-muvesze.html',
   'https://www.origo.hu/kultura/20190922-sara-sandor-halalhir.html',
   'https://www.origo.hu/filmklub/20190524-meghalt-udvardy-anna-az-oscardijas-mindenki-producere.html'],
  'sources': ['Andy Vajna magángyűjteménye',
   'Vertigo Média',
   'Drakulics Filmgyártó Kft./Gerő Elektra',
   'MTI/EPA/Jason Szenes',
   'Szupermodern Filmstúdió'],
  'author': 'Origo',
  'publish_date': '2019.12.31. 13:50'},
 'https://www.origo.hu/itthon/20191231-igy-tud-majd-kozlekedni-szilveszter-ejjel-budapesten.html': {'related_links': ['https://bkk.hu/2019/12/kozlekedes-budapesten-az-ev-vegi-unnepek-idejen-szilveszterkor-egesz-ejszaka-jarnak-a-metrok/'],
  's

In [52]:
df = pd.DataFrame.from_dict(origo_dict, orient = "index").reset_index()